In [1]:
!pip install PySastrawi

In [2]:
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [3]:
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory
factory = StemmerFactory()
stemmer = factory.create_stemmer()

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import numpy as np
import random
import json
import pickle

In [4]:
import pandas as pd
from openpyxl import load_workbook
import datetime

In [5]:
# Load the intents file
with open('intents_edit.json') as data_file:
    intents = json.load(data_file)

In [6]:
# Pre-processing
words = []
classes = []
documents = []
ignore = ['?', '!', '.', ',', ';', ':', '-', '_', '(', ')', '[', ']', '{', '}', '"', "'",
          'dan', 'atau', 'tapi', 'jika', 'sementara', 'dengan', 'tanpa', 'pada', 'di', 'ke',
          'dari', 'oleh', 'untuk', 'dalam', 'yang', 'sebuah', 'adalah', 'adalah', 'ini', 'itu',
          'saya', 'kami', 'kita', 'dia', 'mereka', 'anda', 'aku', 'kamu', 'yang', 'telah',
          'telah', 'akan', 'sedang', 'telah', 'bisa', 'dapat', 'harus']


#tokenize
for intent in intents['intents']:
    for pattern in intent['patterns']:
        w = nltk.word_tokenize(pattern)
        words.extend(w)
        documents.append((w, intent['tag']))
        if intent['tag'] not in classes:
            classes.append(intent['tag'])

In [7]:
# Melakukan stemming dan normalisasi data
words = [stemmer.stem(w.lower()) for w in words if w not in ignore]
words = sorted(list(set(words)))

# Menghapus class duplikat dengan 'set'
classes = sorted(list(set(classes)))

print(len(documents), "documents")
print(len(classes), "classes", classes)
print(len(words), "unique stemmed words", words)

231 documents
39 classes ['Identitas', 'Selamat Tinggal', 'aktivitas', 'apa kabar', 'apresiasi', 'balancing kegiatan', 'balas sapa', 'cuaca', 'delegasi tugas', 'evaluasi kegiatan', 'haha', 'hapus_jadwal', 'istirahat', 'konfirmasi', 'lelucon', 'lihat_jadwal', 'motivasi', 'opsi', 'pembicaraan menyenangkan', 'pengatur waktu', 'pengaturan tujuan', 'pengingat kegiatan', 'perubahan_jadwal', 'prioritas', 'produktivitas', 'programmer', 'proyek', 'rencana bulanan', 'rencana harian', 'rencana mingguan', 'salam', 'seru', 'tambah_jadwal', 'tanggal dan waktu', 'teka-teki', 'terima kasih', 'tidak', 'umur', 'update_jadwal']
219 unique stemmed words ['acara', 'ada', 'aktivitas', 'alarm', 'anda', 'antara', 'apa', 'apakah', 'asah', 'atas', 'atur', 'ayo', 'bagaimana', 'bagi', 'bagus', 'baik', 'banget', 'bantu', 'banyak', 'baru', 'batal', 'batas', 'belum', 'berapa', 'beri', 'besar', 'biasa', 'bicara', 'bikin', 'bisa', 'buang', 'buat', 'bukan', 'bulan', 'butuh', 'bye', 'capai', 'cara', 'cek', 'cerita', 'co

In [8]:
import random
import numpy as np

training = []
output_empty = [0] * len(classes)

for doc in documents:
    pattern_words = [stemmer.stem(word.lower()) for word in doc[0]]
    bag = [1 if w in pattern_words else 0 for w in words]

    output_row = output_empty[:]
    output_row[classes.index(doc[1])] = 1

    training.append([bag, output_row])

random.shuffle(training)
training = np.array(training, dtype=object)

train_x = np.array([item[0] for item in training])
train_y = np.array([item[1] for item in training])


In [9]:
# Reset the default TensorFlow graph
tf.compat.v1.reset_default_graph

# Define the model
model = keras.Sequential([
    layers.InputLayer(input_shape=(len(train_x[0]),), name='input_layer'),
    layers.Dense(10, activation='relu', name='hidden_layer1'),
    layers.Dense(10, activation='relu', name='hidden_layer2'),
    layers.Dense(len(train_y[0]), activation='softmax', name='output_layer')
])

# Compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Set up TensorBoard callback
tensorboard_callback = keras.callbacks.TensorBoard(log_dir='./logs')

# Train the model
model.fit(train_x, train_y, epochs=1000, batch_size=8, callbacks=[tensorboard_callback], verbose=1)

# Save the model
model.save('model.h5')

Epoch 1/1000
29/29 [==============================] - 2s 6ms/step - loss: 3.6604 - accuracy: 0.0476
Epoch 2/1000
29/29 [==============================] - 0s 8ms/step - loss: 3.6412 - accuracy: 0.0563
Epoch 3/1000
29/29 [==============================] - 0s 5ms/step - loss: 3.6222 - accuracy: 0.0563
Epoch 4/1000
29/29 [==============================] - 0s 6ms/step - loss: 3.5992 - accuracy: 0.0476
Epoch 5/1000
29/29 [==============================] - 0s 5ms/step - loss: 3.5695 - accuracy: 0.0476
Epoch 6/1000
29/29 [==============================] - 0s 6ms/step - loss: 3.5316 - accuracy: 0.0476
Epoch 7/1000
29/29 [==============================] - 1s 17ms/step - loss: 3.4895 - accuracy: 0.0476
Epoch 8/1000
29/29 [==============================] - 0s 14ms/step - loss: 3.4360 - accuracy: 0.0649
Epoch 9/1000
29/29 [==============================] - 0s 14ms/step - loss: 3.3727 - accuracy: 0.0736
Epoch 10/1000
29/29 [==============================] - 0s 11ms/step - loss: 3.3006 - accuracy: 0.

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [10]:
# Save
try:
    with open("training_data", "wb") as file:
        pickle.dump({'words': words, 'classes': classes, 'train_x': train_x, 'train_y': train_y}, file)
except IOError as e:
    print(f"Error saving training data: {e}")

# Load
try:
    with open("training_data", "rb") as file:
        data = pickle.load(file)

    words = data['words']
    classes = data['classes']
    train_x = np.array(data['train_x'])
    train_y = np.array(data['train_y'])
except IOError as e:
    print(f"Error loading training data: {e}")

# Load model
try:
    model = keras.models.load_model('model.h5')
except IOError as e:
    print(f"Error loading model: {e}")


In [11]:
def clean_up_sentence(sentence):
    sentence_words = nltk.word_tokenize(sentence)
    sentence_words = [stemmer.stem(word.lower()) for word in sentence_words]
    return sentence_words

def bow(sentence, words, show_details=False):
    sentence_words = clean_up_sentence(sentence)
    bag = np.zeros(len(words), dtype=np.float32)
    for idx, w in enumerate(words):
        if w in sentence_words:
            bag[idx] = 1
            if show_details:
                print(f"Found in bag: {w}")
    return bag

ERROR_THRESHOLD = 0.30

def classify(sentence):
    input_data = bow(sentence, words)
    results = model.predict(np.array([input_data]))[0]
    results = [[i, r] for i, r in enumerate(results) if r >= ERROR_THRESHOLD]
    results.sort(key=lambda x: x[1], reverse=True)
    return [(classes[r[0]], r[1]) for r in results]

def response(sentence, userID='123', show_details=False):
    results = classify(sentence)
    if results:
        for intent in intents['intents']:
            if intent['tag'] == results[0][0]:
                return random.choice(intent['responses'])
    return "Maaf, saya tidak mengerti apa yang Anda katakan."

In [12]:
import pandas as pd
from openpyxl import load_workbook
import datetime
from tabulate import tabulate

In [13]:
def load_schedule():
    file_name="schedule.xlsx"
    try:
        # Cek file excel ada?
        schedule = pd.read_excel(file_name, parse_dates=['Date'])
    except FileNotFoundError:
        #buat df jika tidak ada
        schedule = pd.DataFrame(columns=['Date', 'Start Time', 'End Time', 'Event'])
    return schedule

def save_schedule(file_name, schedule):
    schedule.to_excel(file_name, index=False)

In [14]:
def add_event():
    schedule = load_schedule()
    print("Enter new event details:")
    event_date = input("Date (YYYY-MM-DD): ")
    start_time = input("Start Time (HH:MM): ")
    end_time = input("End Time (HH:MM): ")
    event_name = input("Event Name: ")

    event_date = pd.to_datetime(event_date).date()

    # Conflict?
    for i, row in schedule.iterrows():
        row_date = row['Date'].date() if isinstance(row['Date'], pd.Timestamp) else row['Date']
        if row_date == event_date:
            if (start_time < row['End Time'] and start_time >= row['Start Time']) or \
               (end_time > row['Start Time'] and end_time <= row['End Time']) or \
               (start_time <= row['Start Time'] and end_time >= row['End Time']):
                print(find_nearest_available_time(event_date, start_time, end_time))
                return f"Acaramu yg ini konflik dengan acara : {row['Event']} dari {row['Start Time']} ke {row['End Time']}"


    # Menambahkan event
    new_event = pd.DataFrame([[event_date, start_time, end_time, event_name]],
                             columns=['Date', 'Start Time', 'End Time', 'Event'])
    schedule = pd.concat([schedule, new_event], ignore_index=True)

    # SOrting schedule
    schedule['Date'] = pd.to_datetime(schedule['Date'])
    schedule = schedule.sort_values(by=['Date', 'Start Time']).reset_index(drop=True)
    save_schedule("schedule.xlsx", schedule)
    print("Berhasil menambah skejul")
    print_schedule()


In [15]:
def print_schedule():
    schedule = load_schedule()

    # Add an index column
    schedule.reset_index(inplace=True)
    schedule.rename(columns={'index': 'No'}, inplace=True)

    # Print DataFrame in tabular format
    print(tabulate(schedule, headers='keys', tablefmt='grid', showindex=False))

In [16]:
def update_event():
    schedule = load_schedule()
    print_schedule()
    print("Enter event details to update:")
    index = int(input("Select index [start from 0]: "))
    event_date = input("Date (YYYY-MM-DD): ")
    start_time = input("Start Time (HH:MM): ")
    end_time = input("End Time (HH:MM): ")
    event_name = input("Event Name: ")

    if index >= len(schedule) or index < 0:
        return False, "Invalid index"

    event_date = pd.to_datetime(event_date).date()  # Ubah string 'event_date' menjadi tipe datetime.date
    #start_time = datetime.datetime.strptime(start_time, '%H:%M').time()
    #end_time = datetime.datetime.strptime(end_time, '%H:%M').time()

 # Conflict?
    for i, row in schedule.iterrows():
        if i == index:
            continue
        row_date = row['Date'].date() if isinstance(row['Date'], pd.Timestamp) else row['Date']
        if row_date == event_date:
            if (start_time < row['End Time'] and start_time >= row['Start Time']) or \
               (end_time > row['Start Time'] and end_time <= row['End Time']) or \
               (start_time <= row['Start Time'] and end_time >= row['End Time']):
                print(find_nearest_available_time(event_date, start_time, end_time))
                return f"Acaramu yg ini konflik dengan acara : {row['Event']} dari {row['Start Time']} ke {row['End Time']}"


    # Update event
    schedule.at[index, 'Date'] = event_date
    schedule.at[index, 'Start Time'] = start_time
    schedule.at[index, 'End Time'] = end_time
    schedule.at[index, 'Event'] = event_name

    # Sort schedule by date and start time
    schedule['Date'] = pd.to_datetime(schedule['Date'])
    schedule = schedule.sort_values(by=['Date', 'Start Time']).reset_index(drop=True)
    save_schedule("schedule.xlsx", schedule)
    print("Update berhasil")
    print_schedule()

In [17]:
def delete_event():
    schedule = load_schedule()
    print(print_schedule())
    print("Enter event details to delete:")
    index = int(input("select index [start from 0] : "))
    if index >= len(schedule) or index < 0:
        return False, "Invalid index"
    konfirmasi = input("Anda yakin menghapus jadwal? (0/1) :")
    if konfirmasi == 0 :
        return 'gagal diproses'
    else:
        schedule = schedule.drop(index).reset_index(drop=True)
        save_schedule("schedule.xlsx", schedule)
        print ('proses hapus berhasil')
        print(print_schedule())

In [18]:
def find_nearest_available_time(event_date, start_time, end_time):
    schedule = load_schedule()
    event_date = pd.to_datetime(event_date).date()
    duration_minutes = (datetime.datetime.strptime(end_time, '%H:%M') - datetime.datetime.strptime(start_time, '%H:%M')).total_seconds() / 60
    start_time = datetime.datetime.strptime(start_time, '%H:%M').time()
    end_time = datetime.datetime.strptime(end_time, '%H:%M').time()

    # Convert current event to datetime format
    current_event_start = datetime.datetime.combine(event_date, start_time)
    current_event_end = datetime.datetime.combine(event_date, end_time)

    # Convert 'Start Time' and 'End Time' columns to datetime.time
    schedule['Start Time'] = schedule['Start Time'].apply(lambda x: datetime.datetime.strptime(x, '%H:%M').time())
    schedule['End Time'] = schedule['End Time'].apply(lambda x: datetime.datetime.strptime(x, '%H:%M').time())

    # Sort schedule by date and start time
    schedule['Date'] = pd.to_datetime(schedule['Date'])
    schedule = schedule.sort_values(by=['Date', 'Start Time']).reset_index(drop=True)

    # Check for conflicts
    for i, row in schedule.iterrows():
        row_date = row['Date'].date()
        if row_date == event_date:
            if (current_event_start.time() < row['End Time'] and current_event_start.time() >= row['Start Time']) or \
               (current_event_end.time() > row['Start Time'] and current_event_end.time() <= row['End Time']) or \
               (current_event_start.time() <= row['Start Time'] and current_event_end.time() >= row['End Time']):
                # Conflict found, now look for next available slot
                for j in range(i, len(schedule)):
                    next_start = schedule.at[j, 'End Time']
                    if j == len(schedule) - 1:
                        # Check end of day availability
                        if next_start <= datetime.time(23, 59):
                            next_event_start = next_start
                            next_event_end = (datetime.datetime.combine(event_date, next_event_start) + datetime.timedelta(minutes=duration_minutes)).time()
                            if next_event_end <= datetime.time(23, 59):
                                return f"Waktu konflik dengan acara: {row['Event']} dari {row['Start Time']} hingga {row['End Time']}. Waktu terdekat yang tersedia: {next_event_start} hingga {next_event_end}"
                    else:
                        next_event_start = next_start
                        next_event_end = (datetime.datetime.combine(event_date, next_event_start) + datetime.timedelta(minutes=duration_minutes)).time()
                        if next_event_end <= schedule.at[j + 1, 'Start Time']:
                            return f"Waktu konflik dengan acara: {row['Event']} dari {row['Start Time']} hingga {row['End Time']}. Waktu terdekat yang tersedia: {next_event_start} hingga {next_event_end}"

                # Check before the first event
                if i == 0:
                    if current_event_end <= schedule.at[i, 'Start Time']:
                        return f"Waktu konflik dengan acara: {row['Event']} dari {row['Start Time']} hingga {row['End Time']}. Waktu terdekat yang tersedia: {start_time} hingga {end_time}"

    # If no conflict, return original times
    return "Tidak ada konflik, waktu yang tersedia: {} hingga {}".format(start_time, end_time)


In [19]:
def action(order):
    if order == 'time':
        print(" sekarang ",datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S"))
    if order == 'addschedule':
        add_event()
    if order == 'showschedule':
        print_schedule()
    if order == 'updateschedule':
        update_event()
    if order == 'deleteschedule' :
        delete_event()

In [ ]:
print("0 to close")
while True:
    message = input("")
    if message == "0":
        break
    result = response(message)

    if result is not None and "~" in result:
        order = (result[1:])
        action(order)
    else:
        print(result)


0 to close
1/1 [==============================] - 0s 23ms/step
Enter new event details:
